# Pipeline Tests

## 1. Reader

In [1]:
import pandas as pd
from model.reader import reader

df = reader('train.csv')

In [2]:
TARGET = df.iloc[:, 0]
browser = df.iloc[:, 1]
actions = df.iloc[:, 2:]

## 2. Parser

In [3]:
from model.parser import parse_action_string

parsed_actions = actions.map(parse_action_string)
parsed_actions

,action_1,action_2,action_3,action_4,action_5,action_6,action_7,action_8,action_9,action_10,...,action_14459,action_14460,action_14461,action_14462,action_14463,action_14464,action_14465,action_14466,action_14467,action_14468
0,"(Création d'un écran, infologic.core.accueil.A...","(Affichage d'une dialogue, None, None, None)","(Exécution d'un bouton, None, None, None)","(Fermeture d'une dialogue, None, None, None)","(Affichage d'une dialogue, None, None, None)","(Exécution d'un bouton, None, None, None)","(Fermeture d'une dialogue, None, None, None)","(Création d'un écran, infologic.core.gui.contr...","(t5, None, None, None)","(Exécution d'un bouton, MAINT, None, None)",...,"(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)"
1,"(Création d'un écran, infologic.core.gui.contr...","(Création d'un écran, infologic.core.gui.contr...","(t5, None, None, None)","(Sélection d’un onglet, infologic.orga.modules...","(t10, None, None, None)","(Exécution d'un bouton, None, None, None)","(t15, None, None, None)","(Sélection d’un onglet, None, None, None)","(Exécution d'un bouton, None, None, None)","(Chainage, None, None, None)",...,"(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)"
2,"(Affichage d'une dialogue, infologic.core.gui....","(Exécution d'un bouton, None, None, None)","(Chainage, None, None, None)","(Fermeture d'une dialogue, None, None, None)","(Affichage d'une dialogue, infologic.acti.modu...",(Clic sur une grille d'historique de recherche...,"(Raccourci, None, None, None)","(Fermeture d'une dialogue, None, None, None)","(Création d'un écran, None, None, None)","(t5, None, None, None)",...,"(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)"
3,"(Création d'un écran, infologic.core.gui.contr...","(t5, None, None, None)","(Exécution d'un bouton, MAINT, None, None)","(Affichage d'une dialogue, None, None, None)","(Fermeture d'une dialogue, None, None, None)","(Double-clic, None, None, None)","(Exécution d'un bouton, None, None, None)","(Lancement d'une stat, infologic.core.gui.cont...","(t10, None, None, None)","(Filtrage / Tri, MAINT, None, None)",...,"(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)"
4,"(Affichage d'une dialogue, infologic.acti.modu...","(t5, None, None, None)","(Exécution d'un bouton, None, None, None)","(Fermeture d'une dialogue, None, None, None)","(t10, None, None, None)","(Entrée en saisie dans un formulaire, None, No...","(t10, None, None, None)","(Affichage d'une dialogue, None, None, None)","(t15, None, None, None)","(Fermeture d'une dialogue, None, None, None)",...,"(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)","(None, None, None, None)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3274,"(Création d'un écran, infologic.core.gui.contr...","(Création d'un écran, infologic.core.gui.contr...","(t5, None, None, None)","(Sélection d’un onglet, infologic.orga.modules...","(Saisie dans un champ, None, None, None)","(Retou

## 3. Tokenizer

In [4]:
from model.tokenizer import tokenize_action_sequence

session_tokens, token_to_idx = tokenize_action_sequence(actions)

## 4. Model: Transformer

In [5]:
from model.transformer import create_model, train_model
from sklearn.model_selection import train_test_split

# Create model
vocab_size = len(token_to_idx)
n_usernames = len(TARGET.unique())
model = create_model(vocab_size=vocab_size, n_usernames=n_usernames, max_seq_len=50)

Encode usernames

In [6]:
import torch


unique_usernames = TARGET.unique()
username_to_idx = {username: idx for idx, username in enumerate(unique_usernames)}
TARGET_indices = TARGET.map(username_to_idx).values
TARGET_tensor = torch.tensor(TARGET_indices, dtype=torch.long)

Train Transformer model

In [10]:
# Train
from torch.utils.data import TensorDataset, random_split
import numpy as np

# Use the raw session_tokens (list of lists) instead of converting to tensor
# This allows the training function to handle padding properly

# Split the dataset
train_size = int(0.8 * len(session_tokens))
val_size = len(session_tokens) - train_size

# Split session_tokens and TARGET_tensor
train_tokens = session_tokens[:train_size]
val_tokens = session_tokens[train_size:]

train_targets = TARGET_tensor[:train_size]
val_targets = TARGET_tensor[train_size:]
 
# Train with memory-efficient parameters
train_model(model, (train_tokens, train_targets), (val_tokens, val_targets), 
           epochs=50, batch_size=32, max_seq_len=50)

Training on 2623 samples
Validation on 656 samples
Using max sequence length: 50
Using batch size: 32
Epoch   0: Train Loss: 5.4001, Val Loss: 5.4328, Val Acc: 0.0223
Epoch  10: Train Loss: 5.3582, Val Loss: 5.4159, Val Acc: 0.0223
Epoch  20: Train Loss: 5.3557, Val Loss: 5.4148, Val Acc: 0.0223
Epoch  30: Train Loss: 5.3523, Val Loss: 5.4144, Val Acc: 0.0223


KeyboardInterrupt: 

In [9]:
# Predict username from action sequence
action_sequence = torch.tensor([5, 32, 1, 52, 89])
logits, probs = model.predict_username(action_sequence)
predicted_username = torch.argmax(logits, dim=-1)
predicted_username

tensor([11])